# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Load exported csv file from weatherpy
#delete the unnamed column duplicate the count id



city_df = pd.read_csv('../starter_code/output_data_file')
city_df.dropna(inplace = True) 
del(city_df["Unnamed: 0"])
city_df.head()

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Ushuaia,AR,1651187289,-54.8000,-68.3000,40.66,87,40,25.32
1,Mercedes,AR,1651187035,-34.6515,-59.4307,50.86,47,0,6.29
2,Nadym,RU,1651187290,65.5333,72.5167,22.50,95,100,12.68
3,Punta Arenas,CL,1651187168,-53.1500,-70.9167,42.12,81,98,20.49
4,Hong Kong,HK,1651187165,22.2855,114.1577,81.32,87,28,5.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps.
gmaps.configure(api_key=g_key)

In [12]:
#Use the Lat and Lng as locations and Humidity as the weight.
#get locations data and humidity from the df
locations = city_df[["Latitude","Longitude"]]

humidity =city_df["Humidity"].astype(float)


In [13]:
#Add Heatmap layer to map.
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=80,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig    

    

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_city_df = city_df.loc[(city_df["Max Temperature"] > 70) & 
                               (city_df["Max Temperature"] < 80) & 
                               (city_df["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,Ceres,US,1651187298,37.5949,-120.9577,71.37,39,0,14.97
1,Vila Velha,BR,1651187312,-20.3297,-40.2925,78.75,89,0,4.61
2,Tura,IN,1651187233,25.5198,90.2201,73.26,85,0,4.00
3,Ixtapa,MX,1651187185,20.7000,-105.2000,78.03,78,0,14.97
4,Caravelas,BR,1651187353,-17.7125,-39.2481,75.51,84,0,4.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# set up additional columns to hold information
hotel_df = narrowed_city_df.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

hotel_df.head()


,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Ceres,US,1651187298,37.5949,-120.9577,71.37,39,0,14.97,
1,Vila Velha,BR,1651187312,-20.3297,-40.2925,78.75,89,0,4.61,
2,Tura,IN,1651187233,25.5198,90.2201,73.26,85,0,4.00,
3,Ixtapa,MX,1651187185,20.7000,-105.2000,78.03,78,0,14.97,
4,Caravelas,BR,1651187353,-17.7125,-39.2481,75.51,84,0,4.50,


In [8]:
# find the closest restaurant of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "hotels",
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    #get type from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # add keyword to params dict
    params['location'] = f'{lat},{lng}'

    # assemble url and make API request
    #print(f"Retrieving Results for Index {index}: {row["City"]}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {type} hotels is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        

hotel_df.head()

Closest <class 'type'> hotels is Modesto.
Closest <class 'type'> hotels is Vila Velha.
Closest <class 'type'> hotels is Tura.
Closest <class 'type'> hotels is Puerto Vallarta.
Closest <class 'type'> hotels is Caravelas.
Closest <class 'type'> hotels is Fort Payne.
Closest <class 'type'> hotels is Saint Pierre.
Closest <class 'type'> hotels is Lompoc.
Closest <class 'type'> hotels is Sacramento.
Closest <class 'type'> hotels is Bowmore.
Closest <class 'type'> hotels is Islamkot.
Closest <class 'type'> hotels is Morondava.
Closest <class 'type'> hotels is Khatra.
Closest <class 'type'> hotels is Emerald.


,City,Country,Date,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Ceres,US,1651187298,37.5949,-120.9577,71.37,39,0,14.97,Modesto
1,Vila Velha,BR,1651187312,-20.3297,-40.2925,78.75,89,0,4.61,Vila Velha
2,Tura,IN,1651187233,25.5198,90.2201,73.26,85,0,4.00,Tura
3,Ixtapa,MX,1651187185,20.7000,-105.2000,78.03,78,0,14.97,Puerto Vallarta
4,Caravelas,BR,1651187353,-17.7125,-39.2481,75.51,84,0,4.50,Caravelas


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig


Figure(layout=FigureLayout(height='420px'))